# Resources

In [1]:
import sys
sys.path.append("../../py")
from indirect.problems import orbit2orbit as o2oind
import pygmo as pg
import pygmo_plugins_nonfree as pg7
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
%matplotlib
#%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

Using matplotlib backend: Qt5Agg


# Algorithm

In [ ]:
# algorithm
uda = pg7.snopt7(True, "/usr/lib/libsnopt7_c.so")
uda.set_integer_option("Major iterations limit", 4000)
uda.set_integer_option("Iterations limit", 40000)
uda.set_numeric_option("Major optimality tolerance", 1e-2)
uda.set_numeric_option("Major feasibility tolerance", 1e-8)
algo = pg.algorithm(uda)
#uda = pg.mbh(algo, 1)
#algo = pg.algorithm(uda)

# Indirect: Mass Optimal Orbit to Orbit
$$z = [T, M_0, M_f, \vec{\lambda}_0]$$

In [ ]:
# guess from planet to planet
sol = np.load("../../npy/indirect_planet2planet.npy")
# guess
pert = 0.5
T = sol[1]
M = [1, 3]
l = sol[2:]
guess = np.hstack(([T], M, l))

In [ ]:
# guess from orbit 2 orbit
guess = np.load("../../npy/indirect_orbit2orbit.npy")
pert = 2
T = guess[0]
T

In [ ]:
# mass optimal control
udpind = o2oind(
    mass=1000, tmax=0.3, isp=2500, atol=1e-12, rtol=1e-12,
    Tlb=T-(T*pert), Tub=T+(T*pert)
)
udpind.leg.dynamics.alpha = 1
udpind.leg.dynamics.bound = False
probind = pg.problem(udpind)
udpind.Tlb

In [ ]:
# use previous solution
popind = pg.population(probind, 1)
#popind.push_back(guess)

In [ ]:
# evolve
popind = algo.evolve(popind)

In [ ]:
# plot trajectory
udpind.plot_traj(popind.champion_x)

In [ ]:
# trajectory data
traj = udpind.leg.get_trajectory()

In [ ]:
# plot control
plt.figure()
plt.plot(traj[:,0], traj[:,15], "k-")
plt.xlabel("Time [s]")
plt.ylabel("Throttle [ND]")
plt.show()

In [ ]:
sol = popind.champion_x
sol

In [ ]:
np.save("../../npy/indirect_orbit2orbit.npy", sol)

# Homotopy Parametre

In [ ]:
ntraj = 200
alphas = np.linspace(0,1,ntraj)

In [ ]:
traj = list()

for alpha in alphas:
    
    print(alpha)
    
    # problem
    udpind = o2oind(
        mass=1000, tmax=0.3, isp=2500, atol=1e-10, rtol=1e-10,
        Tlb=T-(T*pert), Tub=T+(T*pert)
    )
    udpind.leg.dynamics.alpha = alpha
    udpind.leg.dynamics.bound = True
    probind = pg.problem(udpind)
    
    # population
    popind = pg.population(probind, 0)
    popind.push_back(guess)
    
    # evolve
    popind = algo.evolve(popind)
    
    # store solution
    guess = popind.champion_x
    
    # set leg
    udpind.fitness(guess)
    
    # store trajectory
    traj.append(udpind.leg.get_trajectory())

In [ ]:
np.save("../../npy/indirect_orbit2orbit_traj.npy", np.asarray(traj))

In [ ]:
trajs = np.load("../../npy/indirect_orbit2orbit_traj.npy")

In [ ]:
plt.figure()
for nt in range(ntraj)[1:]:
    plt.plot(trajs[nt][:,0], trajs[nt][:,15], "k-")
plt.xlabel("Time [s]")
plt.ylabel("Throttle [ND]")
plt.show()

In [ ]:
fig = plt.figure()
axis = fig.gca(projection="3d")
axis.scatter([0], [0], [0], color="y")
for i in range(ntraj)[1:]:
    axis.plot(trajs[i][:,0], trajs[i][:,1], trajs[i][:,2], "k-")
plt.show()

In [ ]:
popind.champion_x

In [3]:
import PyKEP as pk
leg = pk.sims_flanagan.leg()

In [4]:
leg

High-fidelity propagation: 0
Number of segments: 0

Spacecraft mass: 0
Spacecraft thrust: 0
Spacecraft isp: 0
Central body gravitational parameter: 0

Departure date: 2000-Jan-01 00:00:00, mjd2000: 0
Arrival date: 2000-Jan-01 00:00:00, mjd2000: 0
Initial mass: 0 kg
Final mass: 0 kg
State at departure: 0 0 0 0 0 0 0
State at arrival: 0 0 0 0 0 0 0

Throttles values: 

Mismatch at the midpoint: NUMERICAL ERROR!! COULD NOT CALCULATE THE STATE MISMATCH, CHECK YOUR DATA
Throttle magnitude constraints (if negative satisfied): []